Функции активации  в нейронных сетях
====================

Согласно Википедии - В нейронных сетях, функция активации нейрона определяет выход нейрона из входного значения.
Чтобы разобраться, что и как проиходит внутри нейросети, а также как влияет функция активации на выходные значения, следует определить следующие понятия:
* **Входные значения/Inputs/x** - в самом начале, это те, данные на основе которых следует выполнить предсказание для выходного значений.
* **Выходные значения/Целевая переменная/Outputs/y** - те значение, которые необходимо предсказать.
* **Веса/weights/w** - каждому входному х соответствуют какие-то w, которые инициализируются перед обучением, и оптимизируются в процессе обучения. Количество весов для каждого нейрона, зависит от количества x.
* **Отступ/bias/b** - Аналогично весам, инициализируется до обучения и оптимизируется в процессе обучения, но в отличие от весов, bias - это одно число. 
* **Сумматорая функция $S$** - функция вида $xw+b$, где w - weights для входов x, b - bias.
* **Функция активации $A$** - функция, которая применяется к сумматорной, задается до обучения сети, на этапе построения модели.  

Для обычной нейронной сети, с прямым распространением(сигналов(которые на самом деле, какие-то числа)) с 1 нейроном в скрытом слое, последовательность "предсказывания"(вычисления) выходного значения такая:
1. Взять $x$, применить к нему сумматорную функцию, с весами и отступом для скрытого нейрона.
2. К полученному значению функцию активации.
3. Применить к значению из пункта 2 сумматорную функцию, с весами и отступом на выходном слое.
4. Финальный этап: применить функцию активации на выходном слое.  
  
Функция активации играет важнейшую роль, какую именно рассмотрим ниже.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
from ipywidgets import interactive, interact, FloatSlider as FS, HBox, VBox, Dropdown, Tab

%matplotlib inline

Устанавлием параметры графика

In [9]:
rcParams['figure.figsize'] = 13, 6

def interact_hookup(f, controls):
    from ipywidgets import Output
    out = Output()
    def observer(change):
        out.clear_output()
        kwargs = {k:v.value for k,v in controls.items()}
        with out:
            f(**kwargs)
    for k,w in controls.items():
        w.observe(observer, 'value')
    observer(None)
    return out

def slider(v, desc):
    return FS(value=v, min=v-10, max=v+10, description=desc)

def slider_wr(v, desc, rn, rx):
    return FS(value=v, min=rn, max=rx, description=desc)

## Какие бывают функции активации:

**Linear** - обычная линейная функция $y=kx+b$, где $k=w$ - weight(вес нейрона), $b = bias$ - отступ,
формула: $$A(X)=weights*X+bias$$
В какой-то степени, не является активационной функцией вовсе, так как она же и есть сумматорная функция. Исходя из этого, когда **говорят, что функция активации линейная** - это значит функция активации не применяется вовсе. Линейную функцию активации, применяют там, где зависимость между входными данными и целевой переменной линейная или данные линейно разделимы(для задачи классификации).

**Sigmoid(Logistic) a.k.a. Soft step** - функция вида: $$Sigmoid(x)=\frac{1}{1+e^{-x}} $$
Область значений для этой функции $(0;1)$. В сигмойде, для $x<\pm 6$ выходные значения будут: 
$$\begin{cases} 1, & \mbox{если } x > 0 \\ 0, & \mbox{если } x > 0 \end{cases}$$
В случае, если во входных данных $x>\pm 6$, то применяя сигмойду на этих значениях, теряется информативность, в виду того что, они либо обнуляются, либо равны 1. Во избежании этого, необходимо масштабировать входные значения, то есть привести к меньшему промежутку, например разделить на максимальный возможный x из обучающих данных.

**TanH** - [функция гиперболического тангенса](https://en.wikipedia.org/wiki/Hyperbolic_function#Hyperbolic_tangent), в отличии от сигмоиды, может принимать значения на отрезке $(-1;1)$. Для этой функции активации, веса должны быть инициализированы малыми случайными значениями. $$tanh(x) = \frac{2}{1+e^{-2x}} - 1$$
С тангесом, промежуток потери информации, ещё меньше, поэтому необходимо "пошаманить" над данными, также как и с сигмойдой.

**ReLU - Rectified Linear Unit** - дословный перевод: Выпрямленный линейный нейрон. 
$$ReLU(x)=max(0,x)$$
То есть, если $x \in (-\infty , 0)$, то $ReLU(x)=0$, иначе $ReLU(x)=x$  
Наиболее популярная функция активации, однако, имеет ряд недостатков: 
* Недифференцируема в нуле, но диффиринцируемая сколь угодно близко к нулю, но не в самом нуле.
* Нейроны с активацией ReLU могут умереть: случается так, что при определённых значениях весов, выходные значения сети обнуляются, то есть $ReLU(x)=0, \forall x \in Inputs$

#### Почему ReLU популярна, хотя имеет ряд недостатков?
Она хорошо себя показывает на практике, но при обучении, <font color="red">**не следует**</font> устанавливать большой learning rate.
Также, проблемы ReLU, решаются следующими изменениями:

**Elu** - Exponential Linear Unit - улучшенная функция ReLU, записывается она так:
$$ELU(x)= \begin{cases} x, & \mbox{если } x \geq 0 \\ \alpha*(e^x-1), & \mbox{если } x <0 \end{cases}$$
По-умолчанию, $\alpha=1$.

**Leaky ReLU** - функция активации представленная в 2014 году, авторы которой: *Andrew L. Maas, Awni Y. Hannun, Andrew Y. Ng(Очень популярный человек в машинном обучении)*, призванная устранить проблемы ReLU.
$${Leaky \space ReLU}(x)=\begin{cases} x, & \mbox{если } x>0 \\ 0.01x, & \mbox иначе \end{cases}$$

# Функции активации на практике:

In [3]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def tanh(x):
    return 2/(1+np.exp(-2*x)) - 1

def relu(x):
    return np.maximum(x, 0)

def leaky_relu(x):
    return np.array([0.01*v if v<=0 else v for v in x])

def elu(x, alpha=1):
    return np.array([alpha*(np.exp(v)-1) if v<=0 else v for v in x])

## Графики:

Можно выбрать функцию активации из выпадающего списка.

In [4]:
def f(activation=(sigmoid, 'Sigmoid')):
    x = np.linspace(-7,5,100)
    plt.plot(x, activation[0](x), label='{0} activation'.format(activation[1]))
    plt.grid()
    plt.title('{0} function'.format(str(activation[1])))
    plt.xlabel('X')
    plt.ylabel('Output')
    plt.legend()
    plt.show()


interactive_plot = interactive(f,activation={'Sigmoid':(sigmoid, 'Sigmoid'), 'TanH':(tanh,'TanH'),
                                             'ReLU':(relu,'ReLU'), 'ELU':(elu,'ELU'), 
                                             'Leaky ReLU':(leaky_relu,'Leaky ReLU')})
output = interactive_plot.children[-1]
display(interactive_plot)

A Jupyter Widget

Здесь представлен выход Feed-Forward Neural Network с 1 нейроном, то есть:
$$A(S)$$
Ползунки позволяют менять значение w(вес нейрона) и b(bias, отступ нейрона), и посмотреть как изменяется выход сети.

In [5]:
def f(activation=(sigmoid, 'Sigmoid'), w=2, b=1):
    plt.figure(2)
    x = np.linspace(-5, 5, num=1000)
    plt.plot(x, activation[0](x*w+b), label='w={0}, b={1}'.format(w,b))
    plt.grid()
    
    plt.title('{0} neuron'.format(str(activation[1])))
    plt.xlabel('X')
    plt.ylabel('output')
    plt.legend()
    plt.show()

interactive_plot = interactive(f,activation={'Sigmoid':(sigmoid, 'Sigmoid'), 'TanH':(tanh,'TanH'),
                                             'ReLU':(relu,'ReLU'), 'ELU':(elu,'ELU'), 
                                             'Leaky ReLU':(leaky_relu,'Leaky ReLU')},
                                w=(-50.0, 50.0), b=(-10, 10))
output = interactive_plot.children[-1]
interactive_plot

A Jupyter Widget

График аналогичный предыдущему, только теперь в НН 1 скрытый слой с 2-мя нейронами, на выходе требуется 1 значение.

<center> **Таблица для аппроскимации функции $y=x^2$** </center>  

|       | Sigmoid | TanH  |  ReLU |  ELU  | Leaky ReLU |
|:-----:|---------|-------|-------|-------|:----------:|
| $w_1$ |  0.30   | 0.66  |  1.40 | 1.30  |    0.80    |
| $b_1$ | -3.45   | -0.90 | -0.90 | -1.90 |    -0.40   |
| $w_2$ |  0.30   | -0.66 | -1.40 | -1.30 |    -0.80   |
| $b_2$ | -3.45   | -0.90 | -0.90 | -1.90 |    -0.40   |
| $w_3$ | 300.00  | 5.00  |  1.80 | 3.00  |    3.50    |
| $b_3$ |-11.50   | 7.00  |  0.00 | 5.11  |    0.00    |
| $w_4$ | 300.00  | 5.00  |  1.80 | 3.00  |    3.50    |

In [16]:
def f(w1, b1, w2, b2, w3, b3, w4, activation=(sigmoid, 'Sigmoid')):
    
    plt.figure(2)
    
    x = np.linspace(-10, 10, num=1000)
    
    A1, A2 = activation[0](x*w1+b1),  activation[0](x*w2+b2)
    
    A3 = A1*w3 + A2*w4 + b3
    
    plt.plot(x, A1, linestyle='--', label='First neuron activation, w={0}, b={1}'.format(w1,b1))
    plt.plot(x, A2, linestyle='--', label='Second neuron activation, w={0}, b={1}'.format(w2,b2))
    
    plt.plot(x, A1*w3 + b3, linestyle='--', label='First neuron output, w={0}, b={1}'.format(w3,b3))
    plt.plot(x, A2*w4 + b3, linestyle='--', label='Second neuron output, w={0}, b={1}'.format(w4,b3))
    
    plt.plot(x, A3, color='r', linewidth=5.0, label='Output value')
    plt.plot(x, x**2, linewidth=3.0, linestyle=':', label=r'$y=x^2$ function')
    
    plt.grid()
    plt.ylim(-1, 100)
    plt.title('1 hidden layer, with 2 {0} neurons'.format(str(activation[1])))
    plt.xlabel('X')
    plt.ylabel('Output')
    plt.legend()
    plt.show()

acts = Dropdown(options={'Sigmoid':(sigmoid, 'Sigmoid'), 'TanH':(tanh,'TanH'),
                         'ReLU':(relu,'ReLU'), 'ELU':(elu,'ELU'), 
                         'Leaky ReLU':(leaky_relu,'Leaky ReLU')}, 
                          description='Activation: ')
    
params = dict(activation = acts,   w1=slider(0.31,'w1:'), w2=slider(-.31,'w2:'), 
              w3=slider_wr(300,'w3:',-300,300), w4=slider_wr(300,'w4:',-300,300), 
              b1=slider(-3.59,'b1:'), b2=slider(-3.59,'b2:'),  b3=slider(-11.5,'b3:'))  

output = interact_hookup(f, params)
ok = VBox([params['activation'],
           HBox([params['w1'], params['w2']]),  
           HBox([params['w3'], params['w4']]),
           HBox([params['b1'], params['b2']]),
           params['b3'],output])
display(ok)

A Jupyter Widget

Из примера, ясно:
* 2 сигмоидных нейрона, в точности повторяют $x^2$
* ELU засчет изгиба, для $x<0$, очень хорошо справляются с задачей
* TanH требует тонкой настройки параметров
* ReLU и Leaky ReLU справляются с задачей хуже из-за отсутствия нелинейности

## 1 скрытый слой, с 3-мя нейронами

In [6]:
def f(activation=(sigmoid, 'Sigmoid'), **kwargs):
    
    plt.figure(2)
    
    x = np.linspace(-10, 10, num=1000)
    
    A1 = activation[0](x*kwargs['w1']+kwargs['b1'])
    A2 = activation[0](x*kwargs['w2']+kwargs['b2'])
    A3 = activation[0](x*kwargs['w3']+kwargs['b3'])
    
    A4 = A1*kwargs['w4'] + A2*kwargs['w5'] + A3*kwargs['w6'] + kwargs['b4']
    
    plt.plot(x, A1, linestyle='--', label='1st neuron activation')
    plt.plot(x, A2, linestyle='--', label='2nd neuron activation')
    plt.plot(x, A3, linestyle='--', label='3rd neuron activation')
    plt.plot(x, A4, color='r', linewidth=5.0, label='Output value')
    plt.plot(x, np.sin(x), linewidth=3.0, linestyle=':', label=r'$sin(x)$')
    plt.grid()
    plt.ylim(-1, 5)
    plt.title('1 hidden layer, with 3 {0} neurons'.format(str(activation[1])))
    plt.xlabel('X')
    plt.ylabel('Output')
    plt.legend()
    plt.show()

params = dict(activation = acts, w1=slider( 1.35,'w1:'), w2=slider(-1.35,'w2:'), w3=slider(  1, 'w3:'),  
              w4=slider(  1, 'w4:'), w5=slider(  1, 'w3:'),  w6=slider(  1, 'w4:'), b1=slider(-2.5, 'b1:'),
              b2=slider(-2.5, 'b2:'), b3=slider(-1.5, 'b3:'),  b4=slider(-1.5, 'b3:'))  

output = interact_hookup(f, params)
ok = VBox([params['activation'],
           HBox([params['w1'], params['w2'], params['w3']]),  
           HBox([params['w4'], params['w5'], params['w6']]),
           HBox([params['b1'], params['b2'], params['b3'], params['b4']]),
           output])
display(ok)

A Jupyter Widget

## 2 скрытых слоя, по 3 нейрона в каждом 

### $W{_i}{^j}$ - $i$ - слой, $j$ - индекс нейрона

In [7]:
def f(activation=(sigmoid, 'Sigmoid'), **kwargs):
    
    plt.figure(2)
    x = np.linspace(-10, 10, num=1000)
    
    A11 = activation[0](x*kwargs['w11']+kwargs['b11'])
    A12 = activation[0](x*kwargs['w12']+kwargs['b12'])
    A13 = activation[0](x*kwargs['w13']+kwargs['b13'])
    
    A21 = activation[0](A11*kwargs['w21'] + A12*kwargs['w22'] + A13*kwargs['w23'] + kwargs['b21'])
    A22 = activation[0](A11*kwargs['w24'] + A12*kwargs['w25'] + A13*kwargs['w26'] + kwargs['b22'])
    A23 = activation[0](A11*kwargs['w27'] + A12*kwargs['w28'] + A13*kwargs['w29'] + kwargs['b23'])
    
    A3 = kwargs['w31']*A21+kwargs['w32']*A22+kwargs['w33']*A23+kwargs['b3']
    
    plt.plot(x, A11, linestyle='--', label='1st layer 1 neuron activation')
    plt.plot(x, A12, linestyle='--', label='1st layer 2 neuron activation')
    plt.plot(x, A13, linestyle='--', label='1st layer 3 neuron activation')
    
    plt.plot(x, A21, linestyle='--', label='2nd layer 1 neuron activation')
    plt.plot(x, A22, linestyle='--', label='2nd layer 2 neuron activation')
    plt.plot(x, A23, linestyle='--', label='2nd layer 3 neuron activation')
    
    plt.plot(x, A3, color='r', linewidth=5.0, label='Model output value')
    
    plt.plot(x, x**2, linewidth=3.0, linestyle=':', label='sqr(x) function')
    plt.grid()
    plt.ylim(-1, 5)
    plt.title('Model with 2 hidden layer, each have 3 {0} neurons'.format(activation[1]))
    plt.xlabel('X')
    plt.ylabel('output')
    plt.legend()
    plt.show()

params = dict(activation = acts, w11=slider(1,'w11'),w12=slider(1,'w12'),w13=slider(1,'w13'),
              w21=slider(1,'w21'),w22=slider(1,'w22'),w23=slider(1,'w23'),w24=slider(1,'w24'),
              w25=slider(1,'w25'),w26=slider(1,'w26'),w27=slider(1,'w27'),w28=slider(1,'w28'),
              w29=slider(1,'w29'),w31=slider(1,'w31'),w32=slider(1,'w32'),w33=slider(1,'w33'),
              b11=slider(1,'b11'),b12=slider(1,'b12'),b13=slider(1,'b13'),b21=slider(1,'b21'),
              b22=slider(1,'b22'),b23=slider(1,'b23'),b3=slider(1,'b3')) 

tab = Tab()
tab.children = [VBox([HBox([params['w11'],params['w12'],params['w13']]),
                      HBox([params['b11'],params['b12'],params['b13']])]),
                VBox([HBox([params['w21'],params['w22'],params['w23'],params['w24']]),
                      HBox([params['w25'],params['w26'],params['w27'],params['w28']]),
                      HBox([params['w29'],params['b21'],params['b22'],params['b23']])]),
                HBox([params['w31'],params['w32'],params['w33'],params['b3']])]

for i in range(len(tab.children)):
    tab.set_title(i, "{0} layer parameters".format(str(i+1)))

output = interact_hookup(f, params)
ok = VBox([params['activation'], tab, output])
display(ok)

A Jupyter Widget

# Заключение:  
Нейронная сеть, может всё. Но следует тщательно подбирать активационную функцию. Также необходимо проводить предобработку данных, перед обучением.


<div style="text-align: right">**made by <font color="orange">[@iamlion12](https://t.me/iamlion12)</font>**</div>